In [1]:
import pandas as pd
data=pd.read_csv("C:\\Users\\Aarav Menon\\OneDrive\\Desktop\\project\\journal datastd\\DET_EXPRESS2009_3_4_5数据-03.csv")
print(data.head())

   LINK_ID                 TIME  FLOW  LANE_NO  OCC  ASPEED  LARGE_CAR_FLOW
0     3036  2009-06-01 00:01:13     6       13    1      77               0
1     3036  2009-06-01 00:01:13    13       12    2      83               2
2     3036  2009-06-01 00:01:13    16       11    2      68               1
3     3036  2009-06-01 00:01:13     9        3    1      79               0
4     3036  2009-06-01 00:01:13    19        2    4      47               3


In [2]:
print(data.isnull().sum())

LINK_ID           0
TIME              0
FLOW              0
LANE_NO           0
OCC               0
ASPEED            0
LARGE_CAR_FLOW    0
dtype: int64


In [3]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
numerical_features=['FLOW','OCC','ASPEED','LARGE_CAR_FLOW']
data[numerical_features]=scaler.fit_transform(data[numerical_features])
data

,LINK_ID,TIME,FLOW,LANE_NO,OCC,ASPEED,LARGE_CAR_FLOW
0,3036,2009-06-01 00:01:13,-1.184004,13,-0.658748,1.161728,-0.705976
1,3036,2009-06-01 00:01:13,-0.791068,12,-0.567531,1.523713,-0.270084
2,3036,2009-06-01 00:01:13,-0.622667,11,-0.567531,0.618751,-0.488030
3,3036,2009-06-01 00:01:13,-1.015603,3,-0.658748,1.282390,-0.705976
4,3036,2009-06-01 00:01:13,-0.454266,2,-0.385096,-0.648197,-0.052137
...,...,...,...,...,...,...,...
67786,3036,2009-06-22 14:20:26,0.612274,2,-0.111444,0.015442,-0.488030
67787,3036,2009-06-22 14:20:26,0.892942,1,0.070991,0.256766,-0.052137
67788,3036,2009-06-22 14:20:26,0.443873,13,0.527078,-0.225881,1.037594
67789,3036,2009-06-22 14:20:26,1.285878,12,0.618295,0.136104,0.819648


In [4]:
#removing date and other columns
data.drop(columns=['LANE_NO'],inplace=True)
data.drop(columns=['TIME'],inplace=True)
data.drop(columns=['LINK_ID'],inplace=True)
data

,FLOW,OCC,ASPEED,LARGE_CAR_FLOW
0,-1.184004,-0.658748,1.161728,-0.705976
1,-0.791068,-0.567531,1.523713,-0.270084
2,-0.622667,-0.567531,0.618751,-0.488030
3,-1.015603,-0.658748,1.282390,-0.705976
4,-0.454266,-0.385096,-0.648197,-0.052137
...,...,...,...,...
67786,0.612274,-0.111444,0.015442,-0.488030
67787,0.892942,0.070991,0.256766,-0.052137
67788,0.443873,0.527078,-0.225881,1.037594
67789,1.285878,0.618295,0.136104,0.819648


In [6]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input

# Assuming 'data' is your DataFrame containing the preprocessed dataset

# Selecting the features and target variable
X = data[['OCC', 'ASPEED', 'LARGE_CAR_FLOW']]  # Features
y = data['FLOW']  # Target variable

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Reshape features for LSTM (assuming a timestep of 1 hour)
X_reshaped = X_scaled.reshape(X_scaled.shape[0], 1, X_scaled.shape[1])

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size=0.2, random_state=42)

# Build LSTM model
model = Sequential()
model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))  # Use Input(shape) object as the first layer
model.add(LSTM(units=64))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Evaluate the model
mse = model.evaluate(X_test, y_test)
print("Mean Squared Error (LSTM):", mse)

# Make predictions
y_pred = model.predict(X_test)


Epoch 1/10
1695/1695 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.3887
Epoch 2/10
1695/1695 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.1096
Epoch 3/10
1695/1695 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.1026
Epoch 4/10
1695/1695 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.1037
Epoch 5/10
1695/1695 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.1030
Epoch 6/10
1695/1695 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.1018
Epoch 7/10
1695/1695 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.0993
Epoch 8/10
1695/1695 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.0970
Epoch 9/10
1695/1695 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0958
Epoch 10/10
1695/1695 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.0970
424/424 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0914
Mean Squared Error (LSTM): 0.09370400756597519
424/424 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
